In [14]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
from dollarpy import Recognizer, Template, Point
# import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
from collections import defaultdict
from queue import Empty
import base64
import json

mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [13]:

with open('Exercises_Data_whole.csv', 'w' , newline = '') as f:
        writer = csv.writer(f)
        row = ['label' , 'trajectory']
        writer.writerow(row)


In [8]:

def CreatePoints(videoURL,label):
    cap = cv2.VideoCapture(videoURL)
    L_All_point = {}
    L_selected_points = [11,12,13,14,15,16]
    for s in L_selected_points:
        L_All_point[s] = []
    L_train_points = []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        #List to hold Coordinates
        points = []
        f = {}
    
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == False:
                break
            if ret==True:
              
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False        

                results = holistic.process(image)

                image.flags.writeable = True   
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                         )


                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                         )


                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                         )

                try:

                    pose = results.pose_landmarks.landmark
                    index = 0
                    newlist=[]
                    height, width, _ = frame.shape
                    for s in L_All_point:
                        
                        x_cor = int(results.pose_landmarks.landmark[s].x * width)
                        y_cor =  int(results.pose_landmarks.landmark[s].y * height)
                        p_s = [x_cor, y_cor]
                        '''
                        L_All_point[s].append(p_s)
                        '''
                        L_train_points.append(p_s)

                except:
                    pass
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()
    '''
    t = []
    for f in L_All_point:
        t += L_All_point[f]
    points = t
    print(points) 
    traj = json.dumps(points)
    '''
    traj = json.dumps(L_train_points)
    with open('Exercises_Data_whole.csv', 'a+' , newline = '') as f:
        writer = csv.writer(f)
        row = [label , traj]
        writer.writerow(row)
        

In [18]:
vid = "tricep_extension.mp4"

points = CreatePoints(vid,"tricep_extension")
 

In [19]:
#Reading Phase
traj_info = pd.read_csv('Exercises_Data_whole.csv')
map_traj = {}
for index, row in traj_info.iterrows():
      label = str(row['label'])
      trajectories = eval(row['trajectory'])
      trajectories = np.array(trajectories)
      print(label)
      map_traj[label] = trajectories
print(map_traj['lateral_raise'])

bicep_curl
lateral_raise
push_ups
tricep_extension
[[610 243]
 [451 261]
 [698 323]
 ...
 [449 321]
 [594 424]
 [410 401]]


In [20]:
def dollar_Templates(map_trajectories):
  Templates_dollar_py = []
  for k in map_traj:
    curr_traj = map_trajectories[k]
    L = []
    for c in curr_traj:
        p = Point(c[0] , c[1])
        L.append(p)
    template_dpy = Template(k , L)
    print(k)
    print(template_dpy)
    Templates_dollar_py.append(template_dpy)
  return Templates_dollar_py
temps = dollar_Templates(map_traj)
recognizer = Recognizer(temps)

bicep_curl
[(567, 357), stroke None, (439, 373), stroke None, (656, 643), stroke None, (453, 630), stroke None, (541, 909), stroke None, (362, 750), stroke None, (567, 359), stroke None, (440, 372), stroke None, (655, 646), stroke None, (457, 630), stroke None, (541, 910), stroke None, (377, 761), stroke None, (567, 361), stroke None, (442, 372), stroke None, (652, 651), stroke None, (459, 632), stroke None, (537, 920), stroke None, (381, 774), stroke None, (566, 362), stroke None, (444, 371), stroke None, (649, 659), stroke None, (461, 635), stroke None, (533, 936), stroke None, (381, 778), stroke None, (566, 363), stroke None, (446, 371), stroke None, (649, 659), stroke None, (462, 635), stroke None, (533, 936), stroke None, (382, 782), stroke None, (566, 364), stroke None, (447, 370), stroke None, (649, 659), stroke None, (462, 634), stroke None, (533, 934), stroke None, (383, 782), stroke None, (566, 364), stroke None, (449, 370), stroke None, (649, 659), stroke None, (463, 634), s

In [22]:
def Test_dollar_py(rec):
    cap = cv2.VideoCapture('tricep_extension.mp4')
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    size = (frame_width, frame_height)
    out = cv2.VideoWriter('Experiment.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, size)
    L2_point = []
    L_selected_points =  [11,12,13,14,15,16]
    L_test_point = []
    m = {}
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:    
        ct = 0
        while(True):
                ct += 1
                ret, frame = cap.read()
            
                if frame is None:
                    break
            
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False        
                results = holistic.process(image)
                image.flags.writeable = True   
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                height, width, _ = frame.shape
                for f in L_selected_points:                  
                        x_cor = results.pose_landmarks.landmark[f].x
                        y_cor =  results.pose_landmarks.landmark[f].y 
                        x_cor = int(results.pose_landmarks.landmark[f].x * width)
                        y_cor =  int(results.pose_landmarks.landmark[f].y * height)
                        p = Point(x_cor, y_cor)
                        L_test_point.append(p)  
                
                if(ct % 3 == 0):
                    sub = L_test_point.copy()
                    result = rec.recognize(sub)
                    print(result)
                    res_str = result[0]
                    perc_str = str(round(result[1],3))
                    final_res = ""
                    
                    if(res_str is not None):
                        res_str = 'the label is ' + res_str
                        perc_str = 'with ' + perc_str 
                    if(res_str is None):
                        res_str = 'the label is ' + 'None'
                        perc_str = 'with ' + '0'

                    font = cv2.FONT_HERSHEY_SIMPLEX
                    org = (50, 50)
                    fontScale = 1
                    color = (255, 0, 0)
                    thickness = 1
                    cv2.putText(frame, res_str, org, font, 
                            fontScale, color, thickness, cv2.LINE_AA)  
                    org2 = (50 , 100)
                    cv2.putText(frame, perc_str, org2, font, 
                            fontScale, color, thickness, cv2.LINE_AA)  
                    
                out.write(frame)
                cv2.imshow('Show',frame)       
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break


    cap.release()
    cv2.destroyAllWindows()
 
Test_dollar_py(recognizer)


(None, 0)
(None, 0)
(None, 0)
(None, 0)
(None, 0)
(None, 0)
(None, 0)
(None, 0)
('tricep_extension', 0.03996856745675914)
('tricep_extension', 0.421791932894353)
('tricep_extension', 0.3543700378729633)
('tricep_extension', 0.5226818328466752)
('tricep_extension', 0.4257215685339646)
('tricep_extension', 0.36132087368639587)
('tricep_extension', 0.4887208587344831)
('tricep_extension', 0.4810384717424738)
('tricep_extension', 0.24423158927468513)
('tricep_extension', 0.40551752099992167)
('tricep_extension', 0.28817412606104853)
('push_ups', 0.12207591272736329)
('tricep_extension', 0.15883963136712875)
('tricep_extension', 0.2744604171545063)
('tricep_extension', 0.2518767848823732)
('push_ups', 0.31556128450543175)
('tricep_extension', 0.43549673629678565)
('tricep_extension', 0.32189939471148254)
('tricep_extension', 0.2446760526168298)
('tricep_extension', 0.3531074179257617)
('tricep_extension', 0.4683829197744461)
('push_ups', 0.16959638691222012)
('tricep_extension', 0.365799539

In [5]:
import cv2
import numpy as np

def nothing(x):
    pass

# Load image
image = cv2.imread('Ball_last_trial.png')

# Create a window
cv2.namedWindow('image')

# Create trackbars for color change
# Hue is from 0-179 for Opencv
cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
cv2.createTrackbar('VMax', 'image', 0, 255, nothing)

# Set default value for Max HSV trackbars
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize HSV min/max values
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

while(1):
    # Get current positions of all trackbars
    hMin = cv2.getTrackbarPos('HMin', 'image')
    sMin = cv2.getTrackbarPos('SMin', 'image')
    vMin = cv2.getTrackbarPos('VMin', 'image')
    hMax = cv2.getTrackbarPos('HMax', 'image')
    sMax = cv2.getTrackbarPos('SMax', 'image')
    vMax = cv2.getTrackbarPos('VMax', 'image')

    # Set minimum and maximum HSV values to display
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])

    # Convert to HSV format and color threshold
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Print if there is a change in HSV value
    if((phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax) ):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (hMin , sMin , vMin, hMax, sMax , vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax

    # Display result image
    cv2.imshow('image', result)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

(hMin = 0 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 4 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 18 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 27 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 37 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 46 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 56 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 61 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 68 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 71 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 73 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 75 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 76 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = 77 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax =

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2576: error: (-27:Null pointer) NULL window: 'image' in function 'cvGetTrackbarPos'
